In [ ]:
import os

os.chdir("../")

In [ ]:
from src.initialization import credential_init

credential_init()

## Perplexity Connection

https://python.langchain.com/docs/integrations/chat/perplexity/

- sonar-deep-research
- sonar-reasoning-pro
- sonar-reasoning
- sonar-pro
- sonar	128k
- r1-1776

In [ ]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

# model = ChatPerplexity(
#     temperature=0, pplx_api_key=os.environ['PERPLEXITY_API_KEY'], model="r1-1776"
# )

In [ ]:
# system = "You are a helpful assistant."
# human = "{input}"
# prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# pipeline = prompt | model
# response = pipeline.invoke({"input": "Why is the Higgs Boson important?"})
# response.content

### DeepSeek-R1

In [ ]:
import json

from openai import OpenAI

messages = [
    {
        "role": "system",
        "content": (
            "You are an AI assistant that focuses on equity market analysis and you need to "
            "engage in an accurate, comprehensive, helpful and  polite conversation with a user."
        ),
    },
    {  
        "role": "user",
        "content": (
            "Give me a market wrap on the Hong Kong market for Feb 2025."
        ),

    },

]

 
client = OpenAI(api_key=os.environ['PERPLEXITY_API_KEY'], base_url="https://api.perplexity.ai")

 

# chat completion without streaming

response = client.chat.completions.create(

    model="r1-1776",

    messages=messages,

)

print(response)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
# chat completion with streaming

response_stream = client.chat.completions.create(

    model="r1-1776",

    messages=messages,

    stream=True,

)

 

for response in response_stream:

    print('\n ',response)

### Sonar-Deep-Research

In [ ]:
response.citations

In [ ]:
response = client.chat.completions.create(

    model="sonar-deep-research",

    messages=messages,

)

print(response)

## R1-Connection

In [ ]:
from langchain_deepseek import ChatDeepSeek

In [ ]:
model = ChatDeepSeek(api_key=os.environ['DEEPSEEK_API_KEY'], temperature=0, model='deepseek-reasoner')

In [ ]:
system = "You are a helpful assistant."
human = "{input}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

pipeline = prompt | model
response = pipeline.invoke({"input": "Why is the Higgs Boson important?"})
response.content

In [ ]:
print(response.content)

In [ ]:
response

## Combining the Deep Search of Perplexity and Reasoning of DeepSeek-R1 

### Langchain Framework

In [ ]:
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.runnables import chain

@chain
def prompt_template_2_messages(chat_prompt):

    output_messages = []
     
    _messages = chat_prompt.messages

    for message in _messages:
        if isinstance(message, SystemMessage):
            output_messages.append({"role": "system", "content": message.content})
        if isinstance(message, HumanMessage):
            output_messages.append({"role": "user", "content": message.content})

    return output_messages

@chain
def messages_2_perplexity(messages):

    client = OpenAI(api_key=os.environ['PERPLEXITY_API_KEY'], base_url="https://api.perplexity.ai")

    response = client.chat.completions.create(
        model="sonar-deep-research",
        messages=messages
    )

    content = response.choices[0].message.content
    citations = response.citations

    return {"content": content,
            "citations": citations}

In [ ]:
system_prompt = PromptTemplate(template=("You are an AI assistant that focuses on equity market analysis and you need to "
                                "engage in an accurate, comprehensive, helpful and  polite conversation with a user."
                               ))
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="find out the report earnings of Bloomberg ticker {ticker}",
                              input_variable=["ticker"])
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt_perplexity = ChatPromptTemplate.from_messages([system_message,
                                                 human_message
                                               ])

# pipeline = chat_prompt|prompt_template_2_messages


In [ ]:
pipeline_perplexity = chat_prompt_perplexity|prompt_template_2_messages|messages_2_perplexity

In [ ]:
# pipeline_perplexity.invoke({"ticker": '2382 HK'})

### DeepSeek-R1 Pipeline

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

model_deepseek = ChatDeepSeek(api_key=os.environ['DEEPSEEK_API_KEY'], temperature=0, model='deepseek-reasoner')

system_prompt = PromptTemplate(template="You are a helpful assistant.")
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="Create a financial report of {ticker} based on:\n {context}",
                              input_variable=["context", "ticker"])
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt_deepseek = ChatPromptTemplate.from_messages([system_message,
                                                         human_message
                                                        ])

pipeline_deepseek = chat_prompt_deepseek|model_deepseek|output_parser

In [ ]:
from operator import itemgetter


final_pipeline = RunnablePassthrough.assign(context=pipeline_perplexity|itemgetter("content"))|pipeline_deepseek

In [ ]:
output = final_pipeline.invoke({"ticker": '2382 HK'})

In [ ]:
print(output)